In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator
from skimage import io

import tensorflow as tf
from tensorflow import keras

from sklearn.feature_extraction import image

#import kerastuner as kt

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras import regularizers

from data_augmentation import *
from load_helpers import *

In [2]:
"""
from google.colab import drive
drive.mount('/content/drive')
"""

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [ ]:
# Loaded a set of images
root_dir =os.path.join("drive","My Drive","Colab Notebooks","data", "training")

image_dir = os.path.join(root_dir, "images/")
files_im = os.listdir(image_dir)
imgs = np.asarray([load_image(image_dir + files_im[i]) for i in range(len(files_im))])

gt_dir =  os.path.join(root_dir, "groundtruth/")
files_gt = os.listdir(gt_dir)
gt_imgs = np.asarray([load_image(gt_dir + files_gt[i]) for i in range( len(files_gt))])

In [3]:
#run it only once !!!!!!!!! 
#if you wanna run again delete  9dom augmented files!!!!!!!

max_iters=30
augment_dataset(imgs,gt_imgs,image_dir,gt_dir,max_iters)


In [4]:
# Loaded a set of images
files_im = os.listdir(image_dir)
imgs = np.asarray([load_image(image_dir + files_im[i]) for i in range(len(files_im))])

files_gt = os.listdir(gt_dir)
gt_imgs = np.asarray([load_image(gt_dir + files_gt[i]) for i in range( len(files_gt))])

In [8]:
n = len(imgs)

# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [9]:
# Compute features for each image patch
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
X = np.asarray([ img_patches[i] for i in range(len(img_patches))])
Y = np.asarray([value_to_class_NN(gt_patches[i],foreground_threshold) for i in range(len(gt_patches))])

In [10]:
#split train test
test_ratio=0.2
random_state=0
X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size=test_ratio, random_state=random_state)

In [12]:
#imlement CNN
INPUT_SHAPE=(X_train.shape[1],X_train.shape[2],X_train.shape[3])

INPUT = keras.layers.Input(shape=INPUT_SHAPE)

pool0=keras.layers.AveragePooling2D(pool_size=(2,2),
                                    strides=(1,1))(INPUT)
                                    
conv1 =keras.layers.Conv2D(filters=128,
                           kernel_size=(5,5),
                           strides=(1,1),
                           activation='relu',
                           padding='same',
                        input_shape=INPUT_SHAPE)(pool0)
pool1 = keras.layers.MaxPool2D(pool_size=(2,2),
                               padding='same')(conv1)
drop1 = keras.layers.Dropout(0.2)(pool1)

conv2 =  keras.layers.Conv2D(filters=256,
                           kernel_size=(5,5),
                           strides=(2,2),
                           activation='relu',
                           padding='same',
                        input_shape=INPUT_SHAPE)(drop1)    
pool2 = keras.layers.MaxPool2D(pool_size=(2,2),
                               padding='same')(conv2)
drop2 = keras.layers.Dropout(0.3)(pool2)


conv3 =  keras.layers.Conv2D(filters=64,
                           kernel_size=(10,10),
                           strides=(2,2),
                           activation='relu',
                           padding='same',
                        input_shape=INPUT_SHAPE)(drop2)  
                        
pool3 = keras.layers.MaxPool2D(pool_size=(2,2),
                               padding='same')(drop2)
drop3 = keras.layers.Dropout(0.3)(pool3)

flat = keras.layers.Flatten()(drop3)

dense1 = keras.layers.Dense(32,
                            kernel_regularizer=tf.keras.regularizers.l1(0.01),
                            activation='softmax')(flat)

dense2 = keras.layers.Dense(2,
                            kernel_regularizer=tf.keras.regularizers.l1(0.01),
                            activation='sigmoid')(dense1)

model = keras.Model(inputs=INPUT,outputs=dense2)
print(model.summary())
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 40, 40, 3)]       0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 39, 39, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 39, 39, 128)       9728      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 20, 20, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 256)       819456    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 256)        

In [13]:
model.fit(X_train, Y_train, epochs=3, batch_size=64)

Epoch 1/3
185/185 [==============================] - 186s 1s/step - loss: 2.6985 - accuracy: 0.6590
Epoch 2/3
185/185 [==============================] - 185s 1000ms/step - loss: 0.7665 - accuracy: 0.6610
Epoch 3/3
185/185 [==============================] - 189s 1s/step - loss: 0.7523 - accuracy: 0.6610


In [14]:
test=model.evaluate(X_test,Y_test)

93/93 [==============================] - 11s 118ms/step - loss: 0.7551 - accuracy: 0.6591
